In [28]:
import os 
from dotenv import load_dotenv
import tweepy
import pandas as pd
import re
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

/Users/tejus/.pyenv/versions/3.8.5/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
load_dotenv()
# Replace with your keys
b = os.getenv("BEARER_TOKEN")

client = tweepy.Client(bearer_token=b)

In [31]:
def fetch_tweets(max_results=100):
    response = client.search_recent_tweets(query="#Jaguar", max_results=max_results, tweet_fields=['created_at', 'text', 'author_id'])
    tweets = [{'text': tweet.text, 'created_at': tweet.created_at, 'author_id': tweet.author_id} for tweet in response.data]
    return pd.DataFrame(tweets)

tweets_df = fetch_tweets()

print(tweets_df.head())

TooManyRequests: 429 Too Many Requests
Usage cap exceeded: Monthly product cap

In [17]:
def clean_tweet(text):
    text = re.sub(r'@[\w]+', '', text)  # Remove mentions
    text = re.sub(r'#', '', text)       # Remove hashtag symbol
    text = re.sub(r'http\S+', '', text) # Remove URLs
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra whitespace
    return text

tweets_df['cleaned_text'] = tweets_df['text'].apply(clean_tweet)



In [18]:
from langdetect import detect

def detect_language(text):
    try:
        return detect(text)
    except:
        return "unknown"

# Apply language detection
tweets_df['detected_lang'] = tweets_df['cleaned_text'].apply(detect_language)

# Filter for English tweets
english_tweets_df = tweets_df[tweets_df['detected_lang'] == 'en']


In [26]:
english_tweets_df

,text,created_at,author_id,cleaned_text,detected_lang
0,RT @fosgoodwood: The story of the questionable...,2024-12-10 20:05:08+00:00,1253557010831773697,RT : The story of the questionable launch of t...,en
1,"No apologies: @Jaguar's new +$100,000 #luxury ...",2024-12-10 20:04:14+00:00,17463660,"No apologies: 's new +$100,000 luxury EV Call ...",en
2,"#Jaguar #Type00 in #LondonBlue and #MiamiPink,...",2024-12-10 20:02:31+00:00,892086108091293696,"Jaguar Type00 in LondonBlue and MiamiPink, rev...",en
3,"#Jaguar #Type00 in #LondonBlue and #MiamiPink,...",2024-12-10 20:02:02+00:00,2956537286,"Jaguar Type00 in LondonBlue and MiamiPink, rev...",en
4,"#Jaguar #Type00 in #LondonBlue and #MiamiPink,...",2024-12-10 20:01:52+00:00,519823058,"Jaguar Type00 in LondonBlue and MiamiPink, rev...",en
...,...,...,...,...,...
84,What if Jaguar pulls it off?\n\n#jaguar #copyn...,2024-12-10 13:35:08+00:00,1176084287038185474,What if Jaguar pulls it off? jaguar copynothin...,en
86,JAGUAR TYPE 00\nJaguar recently underwent a br...,2024-12-10 13:31:08+00:00,1574302366857236481,JAGUAR TYPE 00 Jaguar recently underwent a bra...,en
92,There's been much discussion around Jaguar's n...,2024-12-10 13:04:52+00:00,16789892,There's been much discussion around Jaguar's n...,en
93,RT @GoodwoodRRC: You already have an opinion o...,2024-12-10 13:02:38+00:00,1260775584478621696,RT : You already have an opinion on Jaguar’s T...,en


BertTweet is a fine tuned version of BERT that is used to sarcasm and "social media language"

In [30]:
model_name = "vinai/bertweet-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/bertweet-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
